In [17]:
import os
import json
import numpy as np
import pandas as pd
from utils.eval import *
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
a = 45; b= 140
mode = "uni"
w_sec = 5; h_sec = w_sec/2


jdata1 = evaluation_single(a, b, mode, "anchor_zero", tolerance=0.13)    # overall acc1 per body segment
jdata2 = evaluation_single(a, b, mode, "anchor_peak", tolerance=0.13)    # overall acc1 per body segment
jdata3 = evaluation_single(a, b, mode, "anchor_energy",tolerance=0.13)    # overall acc1 per body segment

In [ ]:

print("zero-vel")
for key in jdata1["bpm_median"].keys():
    print(key,"---", jdata1["bpm_median"][key]['accuracy'])
    
print("\npeak-vel")
for key in jdata2["bpm_median"].keys():
    print(key,"---", jdata2["bpm_median"][key]['accuracy'])
    
print("\nenergy")
for key in jdata3["bpm_median"].keys():
    print(key,"---", jdata3["bpm_median"][key]['accuracy'])    

### Proposed method

In [11]:
a, b = 45, 140

accuracy_zero = evaluation_multi_segment("anchor_zero", "uni",a,b, tolerance=0.13)
accuracy_peak= evaluation_multi_segment("anchor_peak", "uni",a,b, tolerance=0.13)
accuracy_energy = evaluation_multi_segment("anchor_energy", "uni",a,b, tolerance=0.13)


In [28]:
print("uni")
print("anchor_zero")

for key in accuracy_zero.keys():
    print(key,"---", round(accuracy_zero[key]['acc'],2))
    
print("\nanchor_peak")
for key in accuracy_peak.keys():
    print(key,"---", round(accuracy_peak[key]['acc'],2))    

print("\naccuracy_energy")
for key in accuracy_energy.keys():
    print(key,"---", round(accuracy_energy[key]['acc'],2))    

uni
anchor_zero
bothhand_y_bothfoot_y --- 67.56
leftfoot_xy_rightfoot_xy --- 48.47
left_foot_res_right_foot_res --- 38.18
lefthand_xy_righthand_xy --- 55.48
left_hand_res_right_hand_res --- 39.82
bothfoot_x_bothfoot_y --- 49.07
bothhand_x_bothfoot_x --- 33.04
bothhand_x_bothhand_y --- 60.78
both_hand_res_both_foot_res --- 44.74
bothhand_y_bothfoot_y_torso_y --- 72.93

anchor_peak
bothhand_y_bothfoot_y --- 60.55
leftfoot_xy_rightfoot_xy --- 43.18
left_foot_res_right_foot_res --- 14.62
lefthand_xy_righthand_xy --- 49.66
left_hand_res_right_hand_res --- 14.62
bothfoot_x_bothfoot_y --- 44.37
bothhand_x_bothfoot_x --- 32.29
bothhand_x_bothhand_y --- 55.78
both_hand_res_both_foot_res --- 14.62
bothhand_y_bothfoot_y_torso_y --- 68.31

accuracy_energy
bothhand_y_bothfoot_y --- 66.96
leftfoot_xy_rightfoot_xy --- 47.43
left_foot_res_right_foot_res --- 14.62
lefthand_xy_righthand_xy --- 55.85
left_hand_res_right_hand_res --- 14.62
bothfoot_x_bothfoot_y --- 50.34
bothhand_x_bothfoot_x --- 32.59
bo

## Evaluation: best of N strategy

In [ ]:
segment_names = [["both_hand_y", "both_foot_y"],
                ["both_hand_x", "both_foot_x"],
                ["lefthand_xy", "righthand_xy"],
                ["leftfoot_xy", "rightfoot_xy"],
                ["leftfoot_xy", "rightfoot_xy"],
                ["both_hand_x", "both_hand_y"],
                ["both_foot_x", "both_foot_y"],
                ["both_hand_y", "both_foot_y", "torso_y"]]


In [29]:
a, b = 45, 140
mode = "uni"

# segment_names = ["both_hand_y", "both_foot_y", "torso_y"]
print("best of N")
for comb in segment_names:

    json_data = eval_best_of_n(comb, a, b, mode, "anchor_zero")
    print(comb, "---", round(json_data["accuracy"], 2))
    
    
    
    

best of N
['both_hand_y', 'both_foot_y'] --- 76.44
['both_hand_x', 'both_foot_x'] --- 52.13
['lefthand_xy', 'righthand_xy'] --- 68.75
['leftfoot_xy', 'rightfoot_xy'] --- 61.6
['leftfoot_xy', 'rightfoot_xy'] --- 61.6
['both_hand_x', 'both_hand_y'] --- 74.72
['both_foot_x', 'both_foot_y'] --- 67.11
['both_hand_y', 'both_foot_y', 'torso_y'] --- 85.53


## Body Part Contribution per Dance Genre

In [ ]:
with open("genre_symbols_mapping.json", "r") as file:
    genre_name = json.load(file)


output_path = f"./tempo_estimation_output/tempo_{a}_{b}/"
fpath4 = os.path.join(output_path, f"anchor_zero/left_hand_y_{mode}.pkl")
main_df = pd.read_pickle(fpath4)


# Assuming chosen and main_df have the same length
genre_part_map = defaultdict(list)

for i, (bpm, part) in enumerate(chosen):
    genre = main_df.loc[i, "dance_genre"]
    genre_part_map[genre_name[genre]].append(part)


# Convert to a DataFrame for easier counting
genre_part_df = pd.DataFrame([
    {"genre": genre, "body_part": part}
    for genre, parts in genre_part_map.items()
    for part in parts
])

# Count frequency of each body part per genre
part_counts = (
    genre_part_df.groupby(["genre", "body_part"])
    .size()
    .reset_index(name="count")
    .sort_values(["genre", "count"], ascending=[True, False])
    )

In [ ]:


plt.figure(figsize=(10, 6))
sns.barplot(
    data=part_counts,
    x="count",
    y="genre",
    hue="body_part",
    dodge=True
)

plt.title("Body Part Contribution per Dance Genre")
plt.xlabel("Count")
plt.ylabel("Dance Genre")
plt.legend(title="Body Part", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()
